# create dataset

as this study is based on various datasets, the step of creating dataset is kind of complicated, the steps will be explained with following examples

In [1]:
import sys
sys.path.append('./dataprocesser')
sys.path.append('./synthrad_conversion')
from dataprocesser import step2_create_segmentation as createseg
from dataprocesser import step1_init_data_list as init_data_list

c:\Users\zy7\.conda\envs\torch2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## at first we should have a function to load our customised dataset
for better organizing, we should load our data in form of imagepath + pID
here is an example for synthRAD dataset

here we can try another dataset, namely LITS

In [13]:
import os
from dataprocesser.dataset_LITS import list_img_pID_from_LITS_folder

dir = r'E:\Projects\yang_proj\data\LITS'
LITS_images, LITS_patient_IDs = list_img_pID_from_LITS_folder(dir, isseg=False)
LITS_segs, _ = list_img_pID_from_LITS_folder(dir, isseg=True)
for image, pID in zip(LITS_images, LITS_patient_IDs):
    print(image, pID, '\n')


Found 1 files in E:\Projects\yang_proj\data\LITS 

Found 1 files in E:\Projects\yang_proj\data\LITS 

E:\Projects\yang_proj\data\LITS\volume-0.nii volume-0 



# step 2: create segmentations with the data list using totalsegmentator

here we use the new lits dataset as example

In [5]:
from dataprocesser import step2_create_segmentation as createseg
import torch
import os
gpu_index = 1
device = 'cuda'
task = 'total' # 'total'
GPU_ID=[gpu_index]

# cuda:{GPU_ID[0]}
os.environ["CUDA_VISIBLE_DEVICES"]=f'{gpu_index}'
device = 'cuda' #torch.device(f'cuda' if torch.cuda.is_available() else 'cpu') # 0=TitanXP, 1=P5000

print('use GPU: ', torch.cuda.get_device_name(f'cuda'))
createseg.run(target_file_list = LITS_images, dataset=None, device=device, task=task) 

use GPU:  NVIDIA RTX A6000
create segmentation mask for E:\Projects\yang_proj\data\LITS\volume-0.nii

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Resampling...
  Resampled in 4.00s
Predicting part 1 of 5 ...


c:\Users\zy7\.conda\envs\torch2\Lib\site-packages\nnunetv2\inference\predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(join(

Predicting part 2 of 5 ...


c:\Users\zy7\.conda\envs\torch2\Lib\site-packages\nnunetv2\inference\predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(join(

Predicting part 3 of 5 ...


...

Predicting part 4 of 5 ...


100%|██████████| 27/27 [00:00<00:00, 38.12it/s]


Predicting part 5 of 5 ...


100%|██████████| 27/27 [00:00<00:00, 46.31it/s]


  Predicted in 234.94s
Resampling...
Saving segmentations...
  Saved in 0.38s
segmentation mask is saved as E:\Projects\yang_proj\data\LITS\volume-0_seg.nii


in this way we can get the segmentation as filename_seg.nii.gz that we want

## step 3: create csv file

we import the created datalist to a csv file, so that we can modify and reuse it later

we should get an example.csv with following content:

id,Aorta_diss,seg,img

1PA001,0,E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA001\ct_seg.nii.gz,E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA001\ct.nii.gz

1PA004,0,E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct_seg.nii.gz,E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct.nii.gz

1PA005,0,E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA005\ct_seg.nii.gz,E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA005\ct.nii.gz

# create and save the new LITS dataset

In [17]:
from dataprocesser import create_csv 
output_csv_file = r'tutorial_lits.csv'
print(LITS_images,'\n',LITS_segs, '\n',LITS_patient_IDs)
LITS_Aorta_diss = [0] * len(LITS_images)
datalist_LITS = [[id,Aorta_diss,seg,image] for id,Aorta_diss,seg,image in zip(LITS_patient_IDs, LITS_Aorta_diss, LITS_segs, LITS_images)]
create_csv.create_csv_info_file(datalist_LITS, output_csv_file)

['E:\\Projects\\yang_proj\\data\\LITS\\volume-0.nii'] 
 ['E:\\Projects\\yang_proj\\data\\LITS\\volume-0_seg.nii'] 
 ['volume-0']


# start preparation for training or inference

## create a monai/torch dataloader

this dataloader uses add_CreateContour_MergeMask_MaskHUAssign_transforms, which assigns values to segmentation classes

In [8]:
from synthrad_conversion.utils.my_configs_yacs import init_cfg
from dataprocesser.step1_init_data_list import init_dataset
config_path = 'tutorial_config.yaml'
opt=init_cfg(config_path)
model_name_path='Infer_'+opt.model_name + opt.name_prefix
print(model_name_path)
dataset_name = 'combined_simplified_csv_seg_assigned'
loader, opt, my_paths = init_dataset(opt, model_name_path, dataset_name)
train_loader = loader.train_loader
val_loader  = loader.val_loader
first_batch = next(iter(val_loader))
first_input = first_batch[opt.dataset.indicator_A]
first_target = first_batch[opt.dataset.indicator_B]
print("original image shape:", first_target.shape)


Infer_ddpm2d_seg2med
create combined segmentation dataset with assigned value
create base dataset
combined segmentation assigned dataset use keys: ['source', 'target', 'mask']
scale2000 normalization
scale2000 normalization
use train csv: tutorial.csv
use test csv: tutorial.csv
use keys for creating volume dataset:  ['source', 'target', 'mask']
model name:  ddpm2d_seg2med
val_use_patch:  False
original image shape: torch.Size([1, 1, 512, 512, 75])


In [9]:
# visualize a slice:
import matplotlib.pyplot as plt
def plottorchdata(data, i, slice_id):
    data = data.cpu().numpy().squeeze().squeeze()
    dataimg = f'tutorial_{i}.png'
    plt.figure(i)
    plt.imshow(data[..., slice_id], cmap='gray')
    plt.savefig(dataimg)
slice_id = 50
plottorchdata(first_input, 0, slice_id)
plottorchdata(first_target, 1, slice_id)

# run inference!

In [10]:
import torch
device = torch.device(f'cuda:1' if torch.cuda.is_available() else 'cpu') # 0=TitanXP, 1=P5000
print('use GPU: ', torch.cuda.get_device_name(device))

use GPU:  NVIDIA RTX A6000


In [11]:
from synthrad_conversion.networks.ddpm.ddpm_mri2ct import DiffusionModel
from synthrad_conversion.networks.launch_model import launch_model
import synthrad_conversion.networks as networks
from synthrad_conversion.networks.model_registry import MODEL_REGISTRY
print("all available models:")
for model in MODEL_REGISTRY:
    print(model)
print("use model:", opt.model_name)
mode = "test"
model_name = opt.model_name
train_patient_IDs=loader.train_patient_IDs
test_patient_IDs=loader.test_patient_IDs
launch_model(model_name, opt, my_paths, train_loader, val_loader, mode,
                 train_patient_IDs, test_patient_IDs)


all available models:
ddpm2d_seg2med
spade_ddpm2d_seg2med
pix2pix
cycle_gan
AttentionUnet
resUnet
use model: ddpm2d_seg2med
use pretrained model: logs\241118ddpm_512.pt
continue from epoch 171
continue from step 1783714
aorta dissection not manually set, use the value in csv file
original image shape: torch.Size([1, 1, 512, 512, 75])

 inference val set from 0 to 0 batch


  0%|          | 0/1 [00:00<?, ?it/s]



patient ID: 1BA001
input volume shape: torch.Size([1, 512, 512, 75])
target volume shape: torch.Size([1, 512, 512, 75])
ad: tensor(0)


100%|██████████| 500/500 [04:30<00:00,  1.85it/s]


[Epoch 171] [pID 1BA001] [totalstep 0] [loss: 0.004563] [ssim: 0.940804] [psnr: 23.956906] [mae: 50.321005] 

2025-04-25 21:31:18,373 INFO image_writer.py:197 - writing: logs\20250425_2125_Infer_ddpm2d_seg2med\saved_outputs\volume-0_1BA001_synthesized_0.nii.gz
2025-04-25 21:31:18,422 INFO image_writer.py:197 - writing: logs\20250425_2125_Infer_ddpm2d_seg2med\saved_outputs\volume-0_seg_1BA001_seg_0.nii.gz
2025-04-25 21:31:18,600 INFO image_writer.py:197 - writing: logs\20250425_2125_Infer_ddpm2d_seg2med\saved_outputs\volume-0_1BA001_target_0.nii.gz
[Epoch 171] [pID 1BA001] [totalstep 1] [loss: 0.004563] [ssim: 0.931547] [psnr: 24.208621] [mae: 53.802768] 

2025-04-25 21:31:18,645 INFO image_writer.py:197 - writing: logs\20250425_2125_Infer_ddpm2d_seg2med\saved_outputs\volume-0_1BA001_synthesized_1.nii.gz
2025-04-25 21:31:18,682 INFO image_writer.py:197 - writing: logs\20250425_2125_Infer_ddpm2d_seg2med\saved_outputs\volume-0_seg_1BA001_seg_1.nii.gz
2025-04-25 21:31:18,690 INFO image_wri

100%|██████████| 1/1 [04:52<00:00, 292.99s/it]


# convert all generated slices to volume

In [ ]:
def rename(folder_path):
    ## rename file as the format pID_type_sliceID
    # 1 delete the prefix, prefix: ct, ct_seg, ct_volume
    for filename in os.listdir(folder_path):
        old_file_path = os.path.join(folder_path, filename)
        parts = filename.split('_')
        if len(parts)>=3:
            patient_ID = parts[0]
            sign_name = parts[-2]
            slice_ID = parts[-1]

            if sign_name == 'seg':
                new_filename = f'{patient_ID}_seg_{slice_ID}'
            elif sign_name == 'input':
                new_filename = f'{patient_ID}_seg_{slice_ID}'
            elif sign_name == 'target':
                new_filename = f'{patient_ID}_target_{slice_ID}'
            else: # for synthesized and mask
                new_filename = f'{patient_ID}_{sign_name}_{slice_ID}'
            if not new_filename.endswith('.nii.gz'):
                new_filename += '.nii.gz'
            new_file_path = os.path.join(folder_path, new_filename)
            os.rename(old_file_path, new_file_path)
            print(f'Renamed {old_file_path} to {new_file_path}')

In [ ]:
from seg2med_evaluation import stack_volume_synthrad_anish 
import importlib
importlib.reload(stack_volume_synthrad_anish)  # 强制重新加载模块
root_path = r'logs'
folder_list = [
        '20250403_0359_Infer_ddpm2d_seg2med',
                ]
for folder in folder_list:
    folder_path = os.path.join(root_path, folder, 'saved_outputs')
    # rename(folder_path)
    stack_volume_synthrad_anish.stack_volume(folder_path)